In [ ]:
%load_ext autoreload
%autoreload 2
import copy
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tqdm
from matplotlib.animation import FuncAnimation

import tsam
from tsam import ClusterConfig, SegmentConfig
from tsam.timeseriesaggregation import unstackToPeriods

%matplotlib inline
import math
import multiprocessing
import subprocess

import matplotlib

matplotlib.rcParams["animation.embed_limit"] = 1000

Read in time series from testdata.csv with pandas

Setup the base aggregation configuration

Load the resulting combination

## Create the animated aggregations

In [ ]:
results = results[results["time_steps"] > 80]

In [ ]:
results = pd.concat(
    [results, pd.DataFrame([{"segments": 24, "periods": 365, "time_steps": len(raw)}])],
    ignore_index=True,
)

In [ ]:
results = results.iloc[::-1]

In [ ]:
animation_list = []
previouspredictedPeriods = None
for i, index in enumerate(tqdm.tqdm(results.index)):
    segments = int(results.loc[index, :].to_dict()["segments"])
    periods = int(results.loc[index, :].to_dict()["periods"])

    # aggregate to the selected set using new API
    result = tsam.aggregate(
        raw,
        n_periods=periods,
        period_hours=period_hours,
        cluster=cluster_config,
        segments=SegmentConfig(n_segments=segments),
        rescale=False,
    )

    # and reconstruct the data
    prediction = result.reconstruct()

    # relative reduction of time steps
    reduction = 1 - (float(segments * periods) / len(raw))

    # add a change layer which shows the difference of the latest aggregation to the previous
    if i > 0:
        # difference
        diff_val = previouspredictedPeriods - prediction
        # all fields that changed
        diff_bool = abs(diff_val) > 1e-10
        # make sure that when any change is there it gets set to Nan
        prediction_diff = copy.deepcopy(prediction)
        prediction_diff[diff_bool.max(axis=1)] = np.nan

        # what changes ? segments or periods
        if segments == animation_list[-1]["Segments"]:
            misc = "Clustering periods"
        else:
            misc = "Clustering segments"

        animation_list.append(
            {
                "Prediction": prediction_diff,
                "Segments": segments,
                "Periods": periods,
                "Reduction": reduction,
                "Misc": "Medoid representation",  # misc,
            }
        )

    animation_list.append(
        {
            "Prediction": prediction,
            "Segments": segments,
            "Periods": periods,
            "Reduction": reduction,
            "Misc": "Medoid representation",
        }
    )

    # and set previous prediction periods
    previouspredictedPeriods = prediction

In [ ]:
# Create aggregation with duration representation
result_duration = tsam.aggregate(
    raw,
    n_periods=periods,
    period_hours=period_hours,
    cluster=ClusterConfig(
        method="hierarchical",
        representation="duration",
    ),
    segments=SegmentConfig(n_segments=segments),
    rescale=False,
)

animation_list.append(
    {
        "Prediction": result_duration.reconstruct(),
        "Segments": segments,
        "Periods": periods,
        "Reduction": reduction,
        "Misc": "Distribution representation",
    }
)

Let animation warp - slow in the beginning and slow in the end

Create the plot and the animation loop

And save as animation parelllized with ffmpeg since the default matplotlib implemenation takes too long. Faster implemntation than matplotib from here: https://stackoverflow.com/a/31315362/3253411 

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

In [ ]:
def ani_to_mp4(frame_range, filename):
    canvas_width, canvas_height = fig.canvas.get_width_height()

    # Open an ffmpeg process
    outf = os.path.join("results", filename)
    cmdstring = (
        "ffmpeg",
        "-y",
        "-r",
        "100",  # fps
        "-s",
        f"{canvas_width}x{canvas_height}",  # size of image string
        "-pix_fmt",
        "argb",  # formats
        "-f",
        "rawvideo",
        "-i",
        "-",  # tell ffmpeg to expect raw video from the pipe
        "-vcodec",
        "mpeg4",
        outf,
    )  # output encoding
    p = subprocess.Popen(cmdstring, stdin=subprocess.PIPE)

    # Draw frames and write to the pipe
    for frame in frame_range:
        # draw the frame
        animate(frame)
        fig.canvas.draw()

        # extract the image as an ARGB string
        string = fig.canvas.tostring_argb()

        # write to pipe
        p.stdin.write(string)

    # Finish up
    p.communicate()

In [ ]:
filename_list = os.path.join("results", "filenames.txt")
with open(filename_list, "w") as textfile:
    for filename in filenames:
        textfile.write("file '" + filename + "'\n")

You can also show it inline but it takes quite long.